## The model implementation is insprired from this documentation.
### https://developers.google.com/meridian/notebook/meridian-getting-started#install


In [1]:
# pip install --upgrade google-meridian[colab,and-cuda,schema]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import arviz as az
import IPython
from meridian import constants
from meridian.analysis import analyzer
from meridian.analysis import optimizer
from meridian.analysis import summarizer
from meridian.analysis import visualizer
from meridian.analysis.review import reviewer
from meridian.data import data_frame_input_data_builder
from meridian.data import test_utils
from meridian.model import model
from meridian.model import prior_distribution
from meridian.model import spec
# from meridian.templates import formatter
from schema.serde import meridian_serde
import numpy as np
import pandas as pd
# check if GPU is available
from psutil import virtual_memory
import tensorflow as tf
import tensorflow_probability as tfp

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(
    'Num GPUs Available: ',
    len(tf.config.experimental.list_physical_devices('GPU')),
)
print(
    'Num CPUs Available: ',
    len(tf.config.experimental.list_physical_devices('CPU')),
)

Your runtime has 13.6 gigabytes of available RAM

Num GPUs Available:  1
Num CPUs Available:  1


In [ ]:
file_path = "/content/drive/MyDrive/MMM/model_input2.csv"
df = pd.read_csv(file_path)

In [ ]:
df

,week_start,partner_count,Promo,competitor_change_flag,channel 0_spend,Channel 14_spend,Channel 3_spend,Channel 15_spend,Channel 6_spend,Channel 7_spend,Channel 9_spend,Channel 10_spend,Channel 12_spend,Channel 13_spend,Subscriptions
0,6/27/2022,13,0,0,1111.144573,1774.243165,5974.220364,32926.602486,27807.466802,12234.015385,0.000000,0.000000,0.000000,0.000000,200.164442
1,7/4/2022,15,0,0,1404.464179,5817.086991,13241.295717,20186.001520,23518.138000,43305.798906,0.000000,0.000000,0.000000,0.000000,270.896178
2,7/11/2022,12,1,0,2744.381149,3005.000061,4848.624945,17109.511390,18190.325950,12882.541052,0.000000,0.000000,0.000000,0.000000,232.538114
3,7/18/2022,12,1,0,3493.213503,2103.636901,4264.055469,24322.956580,4081.150901,18703.338647,0.000000,0.000000,0.000000,0.000000,211.083653
4,7/25/2022,15,1,0,3507.389674,0.000000,4362.960291,3549.121942,10996.849154,21550.924570,0.000000,0.000000,5153.713746,0.000000,238.647466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,9/1/2025,30,1,1,10389.694649,3268.539424,42585.660724,34379.760000,60562.245287,56572.388328,13293.683801,25282.749266,28166.097180,21992.566873,1554.034974
167,9/8/2025,19,0,1,7323.204534,4355.950512,48339.157442,30056.199606,53792.262255,37778.786394,8549.283428,28717.060772,20350.098174,18110.431148,1568.748258
168,9/15/2025,31,1,1,8553.947466,4299.617387,27302.938987,39671.203170,7226.720800,34351.025983,8507.231084,24524.977406,14876.060515,45469.049393,900.557577
169,9/22/2025,23,0,1,5389.465949,11005.673136,38716.479320,37814.268530,20065.287024,62514.235068,9427.640100,28477.255222,19211.046113,90457.619187,799.702279


In [ ]:
df['week_start'] = pd.to_datetime(df['week_start'])

# Create the holiday flag
df['holiday_flag'] = df['week_start'].apply(
    lambda d: 1 if ((d.month == 11 and d.day >= 28) or
                    (d.month == 12) or
                    (d.month == 1 and d.day <= 2)) else 0
)

# Check result
# data_compiled[['sei_market_week_start', 'holiday_flag']].head(10)
df.loc[df['holiday_flag']==1]

,week_start,partner_count,Promo,competitor_change_flag,channel 0_spend,Channel 14_spend,Channel 3_spend,Channel 15_spend,Channel 6_spend,Channel 7_spend,Channel 9_spend,Channel 10_spend,Channel 12_spend,Channel 13_spend,Subscriptions,holiday_flag
22,2022-11-28,13,1,0,2331.643713,339.089152,11618.553053,13672.390585,38849.365561,26767.937762,0.000000,0.000000,14036.112334,0.000000,518.037166,1
23,2022-12-05,19,0,0,987.701962,2686.251569,10992.722632,14538.238490,33349.275063,27284.128443,0.000000,0.000000,12311.667770,0.000000,364.427769,1
24,2022-12-12,13,0,0,1845.623379,2626.781167,7297.498094,10858.226050,19482.517408,66413.048320,0.000000,0.000000,11945.363536,0.000000,475.895095,1
25,2022-12-19,12,1,0,1488.064863,1548.295694,10668.946517,6159.460692,14509.169219,66028.329751,0.000000,0.000000,12034.293800,0.000000,261.038525,1
26,2022-12-26,11,0,0,1865.200631,3114.719498,10013.523248,1999.797206,42665.076190,18384.390409,0.000000,0.000000,9925.989792,0.000000,284.145463,1
27,2023-01-02,14,1,0,2226.047627,4029.668503,12385.270707,5453.659127,26095.956810,22482.341746,0.000000,0.000000,6489.642385,0.000000,561.236455,1
75,2023-12-04,22,0,0,2170.406550,0.000000,13150.998405,4755.440416,6038.187305,27559.167535,0.000000,0.000000,0.000000,0.000000,576.961022,1
76,2023-12-11,22,1,0,3049.870694,0.000000,42690.572866,9134.330083,8384.480748,21615.023951,0.000000,0.000000,0.000000,0.000000,383.142986,1
77,2023-12-18,16,0,0,2505.195536,0.000000,36963.025125,5756.986097,13278.537802,20143.748135,0.000000,0.000000,0.000000,0.000000,580.312327,1
78,2023-12-25,15,1,0,1388.889388,0.000000,37462.843457,8695.530402,5491.877152,30759.726759,0.000000,0.000000,0.000000,0.000000,665.288405,1


In [ ]:
channels_to_keep = [
    "Channel 7",
    "Channel 13",
    "Channel 15",
    "Channel 3",
    "Channel 6",
    "Channel 12",
    "Channel 10"
]

# Building the list of spend columns to keep
spend_cols_to_keep = [f"{ch}_spend" for ch in channels_to_keep]

# Listing of spend columns in the dataframe
all_spend_cols = [c for c in df.columns if c.endswith("_spend")]

# Identifying spend columns we DO NOT want
spend_cols_to_drop = [c for c in all_spend_cols if c not in spend_cols_to_keep]

# Final filtered dataframe:
df = df.drop(columns=spend_cols_to_drop)

df.head()


,week_start,partner_count,Promo,competitor_change_flag,Channel 3_spend,Channel 15_spend,Channel 6_spend,Channel 7_spend,Channel 10_spend,Channel 12_spend,Channel 13_spend,Subscriptions,holiday_flag
0,2022-06-27,13,0,0,5974.220364,32926.602486,27807.466802,12234.015385,0.0,0.000000,0.0,200.164442,0
1,2022-07-04,15,0,0,13241.295717,20186.001520,23518.138000,43305.798906,0.0,0.000000,0.0,270.896178,0
2,2022-07-11,12,1,0,4848.624945,17109.511390,18190.325950,12882.541052,0.0,0.000000,0.0,232.538114,0
3,2022-07-18,12,1,0,4264.055469,24322.956580,4081.150901,18703.338647,0.0,0.000000,0.0,211.083653,0
4,2022-07-25,15,1,0,4362.960291,3549.121942,10996.849154,21550.924570,0.0,5153.713746,0.0,238.647466,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   week_start              171 non-null    datetime64[ns]
 1   partner_count           171 non-null    int64         
 2   Promo                   171 non-null    int64         
 3   competitor_change_flag  171 non-null    int64         
 4   Channel 3_spend         171 non-null    float64       
 5   Channel 15_spend        171 non-null    float64       
 6   Channel 6_spend         171 non-null    float64       
 7   Channel 7_spend         171 non-null    float64       
 8   Channel 10_spend        171 non-null    float64       
 9   Channel 12_spend        171 non-null    float64       
 10  Channel 13_spend        171 non-null    float64       
 11  Subscriptions           171 non-null    float64       
 12  holiday_flag            171 non-null    int64     

## Doing a random 80-20 split.
### Can read this documentation for Holdout Observation.
### https://developers.google.com/meridian/docs/advanced-modeling/holdout-observations

#### according to the document -
#### 1. the holdout observation should be balanced across geos. This example is a national model so no need to worry about geos now.
#### 2. the holdout obsrvation should be balanced across time.

#### A random split is good in this case because-
##### (a) prediction of those observation become true out-of-sample predictions.
##### (b) The document warns not to hold out large contnuous blocks, end of series forecast horizons.
##### (c) Works perfectly with knot-based trend model.


In [ ]:
# Random 80/20 split across all weeks
from sklearn.model_selection import train_test_split
train_idx, test_idx = train_test_split(
    df.index,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

df["set"] = "train"
df.loc[test_idx, "set"] = "test"

df_train = df.loc[df["set"] == "train"].copy()
df_test  = df.loc[df["set"] == "test"].copy()

df["set"].value_counts()


,count
set,
train,136
test,35


In [ ]:
# df["set"] already has "train" / "test"

# Meridian expects boolean: True = holdout/test
holdout_bool = (df["set"] == "test").values  # shape (T,)

# Expand to 2D shape (G, T) where G=1
# holdout_id = np.expand_dims(holdout_bool, axis=0)  # shape (1, T)   # should remove for 1 geo
holdout_id = holdout_bool # should use this one for geo 1

print("Holdout shape:", holdout_id.shape)
print("Holdout count:", holdout_id.sum(), "out of", len(df))


Holdout shape: (171,)
Holdout count: 35 out of 171


## 1. Defining KPI, media , controls, non-media

In [ ]:
kpi_col = "Subscriptions"

media_spend_cols = [c for c in df.columns if c.endswith("_spend")]
media_channels = [c.replace("_spend", "") for c in media_spend_cols]

print("Media channels:")
for ch in media_channels:
    print("-", ch)

Media channels:
- Channel 3
- Channel 15
- Channel 6
- Channel 7
- Channel 10
- Channel 12
- Channel 13


### Since there is only spend, set spend columns as media columns

## Meridian needs date column. Going to use week_start.

In [ ]:
df["time"] = pd.to_datetime(df["week_start"])

In [ ]:
media_cols = media_spend_cols


control_cols = [
    "partner_count",
    "holiday_flag"
]

non_media_treatment_cols = ["Promo"]

## 2. Building InputData for Meridian

### Meridian cannnot directly consume a raw pandas Dataframe.
### It needs the data to be structured into a very specific internal format that tells the model:
#### - What the KPI is
#### - what the control varaibles are
#### - what columns are media spend
#### - What columns are media exposure
#### - what are non-media treatments
#### - how time and grouping are defined
#### - how to align shapes and dtypes
#### - how to check for missing or invalid columns
#### - how to validate that everything fits the expected causal structure

In [ ]:
builder = data_frame_input_data_builder.DataFrameInputDataBuilder(
    kpi_type="non_revenue",
    default_kpi_column=kpi_col,
)

builder = (
    builder
    .with_kpi(df)
    .with_controls(df, control_cols=control_cols)
    .with_media(
        df,
        media_cols=media_cols,
        media_spend_cols=media_spend_cols,
        media_channels=media_channels,
    )
    .with_non_media_treatments(
        df,
        non_media_treatment_cols=non_media_treatment_cols,
    )
)

input_data = builder.build()

/usr/local/lib/python3.12/dist-packages/meridian/data/input_data.py:524: UserWarning: Consider setting custom priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(


## 3. Configure the Meridian model (custom prior without revenue_per_kpi)

### By Default Merdian models revenue but in this example we are mdoeling Subscriptions which is a non revenue KPI.

### So cannot use the default prior mean and prior standard deviation given in the example documentation.

### There is another documentation that explains setting prior when the outcome of the model is non-revenue.

### https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom

#### 3.1 Compute KPI and cost

In [ ]:
import numpy as np
import tensorflow_probability as tfp
from meridian import constants
from meridian.model import model, spec, prior_distribution

# Total KPI across all time (across all geos, but there is only one geo)
kpi_total = df[kpi_col].sum()

# toal cost per channel across time
cost_per_channel = df[media_spend_cols].sum(axis=0).values

# sanity check
print("Total KPI (Subscriptions):", kpi_total)
print("Total cost per channel:", cost_per_channel)
print("Total media cost:", cost_per_channel.sum())

Total KPI (Subscriptions): 117791.43963679
Total cost per channel: [3666574.56670044 3873891.35840811 2903364.08682975 6207882.61070553
 1323022.03379572 1512381.21141763 4471629.23639465]
Total media cost: 23958745.10425182


#### 3.2 Setting prior on total paid media contribution

In [ ]:
# Prior belief: ~50% of KPI is from media, ±15%
# stating my belief
p_mean = 0.35   # prior mean proportion of KPI incremental due to ALL media. 50% of subscriptions come from media
p_sd   = 0.25  # prior std dev of that proportion. With uncertainty 15%

#Intuition
# believe that half of all subscriptions come from marketing spend
# but not certain, so allow wiggle room (± 15%).
# This is global. not channel specific

# Now from KPI convert to ROI which is total KPI / total spend.
# This is saying If media generated 50% of KPI, what is the implied IKPC (incremental KPI per dollar)
# Derived ROI/IKPC prior mean & sd
roi_mean = p_mean * kpi_total / np.sum(cost_per_channel)
roi_sd   = p_sd   * kpi_total / np.sqrt(np.sum(np.power(cost_per_channel, 2)))

print("Implied ROI (IKPC) prior mean:", roi_mean)
print("Implied ROI (IKPC) prior sd:", roi_sd)


# Meridian requires ROI priors to be LogNormal (because ROI must be > 0)
# a log normal distribution is defined by: μ (mu) → log-space mean,  σ (sigma) → log-space standard deviation

# # Convert to LogNormal parameters
# lognormal_sigma = np.sqrt(np.log(roi_sd**2 / roi_mean**2 + 1))
# lognormal_mu    = np.log(roi_mean * np.exp(-lognormal_sigma**2 / 2))

lognormal_sigma= float(
    np.sqrt(np.log(roi_sd**2 / roi_mean**2 + 1))
)
lognormal_mu = float(
    np.log(roi_mean * np.exp(-lognormal_sigma**2 / 2))
)


print("LogNormal mu:", lognormal_mu)
print("LogNormal sigma:", lognormal_sigma)

Implied ROI (IKPC) prior mean: 0.0017207497176285824
Implied ROI (IKPC) prior sd: 0.0029528986324761494
LogNormal mu: -7.0511993642605
LogNormal sigma: 1.171498325629885


#### 3.3 Building the Prior Distribution and ModelSpec

In [ ]:
# Gives one LogNormal ROI prior that:
# Encodes prior belief: “media drives 50% of KPI”
# Is calibrated to the data KPI scale
# Is calibrated to the data spend scale
# Is consistent across all media channels

n_channels = len(media_channels)  # should be 12 for you

loc_list   = [lognormal_mu]    * n_channels
scale_list = [lognormal_sigma] * n_channels

print("loc_list len:", len(loc_list))
print("scale_list len:", len(scale_list))

roi_prior = tfp.distributions.LogNormal(
    loc=loc_list,
    scale=scale_list,
    name=constants.ROI_M,
    # dtype=tf.float32,   # <- ensure model math uses float32
)

# for all media channels use this LogNormal ROI prior
prior = prior_distribution.PriorDistribution(
    roi_m=roi_prior
)

model_spec = spec.ModelSpec(
    prior=prior,
    enable_aks=True,            # allows richer prior structure
    holdout_id=holdout_id
    # media_prior_type="roi",   # default
    # rf_prior_type="roi",      # only needed if there is reach/frequency in the data
)

mmm = model.Meridian(input_data=input_data, model_spec=model_spec)


loc_list len: 7
scale_list len: 7


/usr/local/lib/python3.12/dist-packages/meridian/model/model.py:74: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(


## 4. Sample prior and posterior

In [ ]:
# Prior sampling
mmm.sample_prior(500)

# Posterior sampling
mmm.sample_posterior(
    n_chains=6,      # or 8–10 if runtime is acceptable
    n_adapt=1500,    # warmup
    n_burnin=500,
    n_keep=800,
    seed=0,
)


/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_om has been automatically se

### 4.1 getting out-of-sample predictions

### visualizer.ModelFit(mmm) has atribute model_fit_data. This is an xarray dataset that contains-
#### geo, time, metric (mean, ci_hi, ci_lo)
#### data variables-
#### expected (model's predicted outcome)
#### baseline (baseline component) [the portion of the KPI that the model explains without any paid media. It is the model's estimate of what Subscriptions would have been even if you spent $0 on media] given seasonality, long-term trend, control variables, competitor effects, promotions / non-media treatments, geo effects
#### actual (observed KPI)

In [ ]:
from meridian.analysis import visualizer

# 1) Build the ModelFit object
model_fit = visualizer.ModelFit(mmm)

# 2) Get the xarray Dataset with expected/baseline/actual
fit_ds = model_fit.model_fit_data  # xarray.Dataset

# 3) Convert to a tidy pandas DataFrame
pred_df = fit_ds.to_dataframe().reset_index()

pred_df.head()


/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:629: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


,geo,time,metric,expected,baseline,actual
0,national_geo,2022-06-27,mean,268.508789,203.584732,200.164444
1,national_geo,2022-06-27,ci_lo,186.635101,89.274307,200.164444
2,national_geo,2022-06-27,ci_hi,339.262909,297.236359,200.164444
3,national_geo,2022-07-04,mean,293.949646,192.689560,270.896179
4,national_geo,2022-07-04,ci_lo,231.342865,77.421387,270.896179


### going to keep the mean row per time / geo

In [ ]:
pred_mean = pred_df[pred_df["metric"] == "mean"].copy()

# Rename columns for clarity
pred_mean = pred_mean.rename(
    columns={
        "expected": "expected_kpi",
        "actual": "actual_kpi",
    }
)
pred_mean['time'] = pd.to_datetime(pred_mean['time'])

pred_mean.head()


,geo,time,metric,expected_kpi,baseline,actual_kpi
0,national_geo,2022-06-27,mean,268.508789,203.584732,200.164444
3,national_geo,2022-07-04,mean,293.949646,192.689560,270.896179
6,national_geo,2022-07-11,mean,251.959793,209.032318,232.538116
9,national_geo,2022-07-18,mean,249.843445,209.032318,211.083649
12,national_geo,2022-07-25,mean,241.103104,192.689560,238.647461


### Merging predictions back to original df with train / test labels

In [ ]:
df_eval = df.merge(
    pred_mean[["geo", "time", "expected_kpi", "baseline", "actual_kpi"]],
    on=["time"],
    how="left",
)


In [ ]:
df_train_eval = df_eval[df_eval["set"] == "train"].copy()
df_test_eval  = df_eval[df_eval["set"] == "test"].copy()

def mape(actual, pred):
    return np.mean(np.abs((actual - pred) / actual))

kpi_col = "Subscriptions"

print("Train MAPE:", mape(df_train_eval[kpi_col], df_train_eval["expected_kpi"]))
print("Test  MAPE:", mape(df_test_eval[kpi_col],  df_test_eval["expected_kpi"]))


Train MAPE: 0.13640432329994104
Test  MAPE: 0.2482455476944135


## 5. Diagnostics and quality review

In [ ]:
# High-level reviewer checks
review = reviewer.ModelReviewer(mmm).run()
print(review)

# R-hat diagnostics
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

# Model fit: actual vs expected Subscriptions
model_fit = visualizer.ModelFit(mmm)
model_fit.plot_model_fit()


/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:629: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


Model Quality Checks
Overall Status: PASS
Summary: Passed: No major quality issues were identified.

Check Results:
----------------------------------------
Convergence Check:
  Status: PASS
  Recommendation: The model has likely converged, as all parameters have R-hat values < 1.2.
----------------------------------------
Baseline Check:
  Status: PASS
  Recommendation: The posterior probability that the baseline is negative is 0.00. We recommend visually inspecting the baseline time series in the Model Fit charts to confirm this.
----------------------------------------
BayesianPPP Check:
  Status: PASS
  Recommendation: The Bayesian posterior predictive p-value is 0.94. The observed total outcome is consistent with the model's posterior predictive distribution.
----------------------------------------
GoodnessOfFit Check:
  Status: PASS
  Recommendation: R-squared = 0.9044 (All), 0.9177 (Train), 0.8633 (Test); MAPE = 0.1593 (All), 0.1364 (Train), 0.2482 (Test); wMAPE = 0.1197 (All),

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:629: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


alt.LayerChart(...)

### 5.1 rhat

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(


alt.LayerChart(...)

## 6. Summarizer (2-page results, effectiveness, ROAS, decomposition)

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(


In [ ]:
# filepath = '/content/drive/MyDrive/MMM/'

# mmm_summarizer.output_model_results_summary(
#     'summary_output2.html', filepath
# )

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:629: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:629: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:3238: UserWarning: Effectiveness is not reported because it does not have a clear interpretation by time period.
  warnings.warn(


## 7. Save the model

In [ ]:
# file_path = '/content/drive/MyDrive/MMM/saved_mmm2.binpb'
# meridian_serde.save_meridian(mmm, file_path)

load the model

In [ ]:
file_path = '/content/drive/MyDrive/MMM/saved_mmm2.binpb'
loaded_mmm = meridian_serde.load_meridian(file_path)

/usr/local/lib/python3.12/dist-packages/meridian/data/input_data.py:524: UserWarning: Consider setting custom priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/model.py:74: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(


In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(loaded_mmm)
model_diagnostics.plot_rhat_boxplot()

/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_om has been automatically se

alt.LayerChart(...)

## 8. Running Budget Optimizer

In [ ]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(loaded_mmm)
optimization_results = budget_optimizer.optimize()

/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_om has been automatically se

CPU times: user 21.3 s, sys: 1.2 s, total: 22.5 s
Wall time: 25.1 s


In [ ]:
# filepath = '/content/drive/MyDrive/MMM/'
# optimization_results.output_optimization_summary(
#     'optimization_output.html', filepath
# )

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(
